In [1]:
# Automatic reload of local libraries
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [1]:
fdocs = "/home/mrim/data/collection/msmarco/documents/msmarco-docs-irr-words.tsv"
fwords = "/home/mrim/data/collection/msmarco/documents/msmarco-words.tsv"
train_queries = "/home/mrim/data/collection/msmarco/documents/train/msmarco-doctrain-queries-words.tsv"
train_qrels = "/home/mrim/data/collection/msmarco/documents/train/msmarco-doctrain-qrels.tsv"
valid_queries = "/home/mrim/data/collection/msmarco/documents/dev/msmarco-docdev-queries-words.tsv"
valid_qrels =  "/home/mrim/data/collection/msmarco/documents/dev/msmarco-docdev-qrels.tsv"
test_queries = "/home/mrim/data/collection/msmarco/documents/test/msmarco-test2019-queries-words.tsv"
test_qrels = "/home/mrim/data/collection/msmarco/documents/test/2019qrels-docs.txt"
fembeddings =  "/home/mrim/data/embeddings/cc.en.300.bin"
EMB_LEN = 300
QMAX_LEN = 10
DMAX_LEN = 100

In [2]:
from utils.msmarco.data_loader import DataLoader
import utils

docs_dict = utils.msmarco.data_loader.load_docs(fdocs)

Documents are loaded in train_loader


In [3]:
train_loader = utils.msmarco.data_loader.DataLoader(
    train_queries, train_qrels, docs_dict,
)

In [4]:
valid_loader = utils.msmarco.data_loader.DataLoader(
    valid_queries, valid_qrels, docs_dict,
)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import fasttext
import random

class Embeddings:
    def __build_emb_list(self, word_file, emb_file):
        if not self.is_stub:
            self.model = fasttext.load_model(emb_file)
        self.dim = self.model.get_dimension() if not self.is_stub else 300
        word_ids = open(word_file, "r", encoding="utf-8")
        self.word_embeddings = []
        i = 0
        for line in word_ids:
            word_id, word = line.rstrip().split("\t")
            self.word_embeddings.append(
                self.model[word]
                if not self.is_stub
                else np.random.normal(0, 1.0, self.dim)
            )
            assert i == int(word_id)
            i += 1
        print("Word_id to embedding is built", flush=True)

    def __init__(self, emb_file, word_file, is_stub=False):
        self.is_stub = is_stub
        self.__build_emb_list(word_file, emb_file)

    def matrix2(self, texts, max_len):
        matrix = np.zeros((len(texts), max_len, self.dim))
        for t in range(len(texts)):
            words = texts[t].split(" ")
            for i in range(min(len(words), max_len)):
                if words[i] == "":
                    continue
                matrix[t][i] = self.word_embeddings[int(words[i])]

        return matrix


In [6]:
embeddings = Embeddings(fembeddings, fwords, is_stub=False)

Word_id to embedding is built


In [7]:
def build_emb_input(embeddings, batch, qmax_len, dmax_len):
    queries = batch[0]
    docs1 = batch[1]
    docs2 = batch[2]

    q_emb = embeddings.matrix2(queries, max_len=qmax_len)
    doc1_emb = embeddings.matrix2(docs1, max_len=dmax_len)
    doc2_emb = embeddings.matrix2(docs2, max_len=dmax_len)

    return q_emb, doc1_emb, doc2_emb


def reshape2_4d(tensor):
    tensor = np.asarray([tensor[i].transpose() for i in range(tensor.shape[0])])
    return torch.from_numpy(tensor).float()

In [102]:
class Autoencoder(nn.Module):
    def __init__(self, layer_size, dropout_prob=0.8):
        super().__init__()
        self.layer_size = layer_size
        self.fc = nn.ModuleList([])
        
        for i in range(len(layer_size)-1):
            self.fc.append(nn.Conv1d(layer_size[i], layer_size[i+1], 5 if i == 0 else 1, bias=False))
            
        for i in range(len(self.fc)):
            torch.nn.init.normal_(self.fc[i].weight)
        
        self.dropout = nn.Dropout(p=dropout_prob, inplace=False)

    def forward(self, x):
        for i in range(len(self.fc)):
            x = self.dropout(F.relu(self.fc[i](x)))
        x = torch.mean(x, 2, keepdim=True)
        return x

In [83]:
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device to use:", device)
epochs = 30
autoencoder = Autoencoder([EMB_LEN, 500, 10000])
autoencoder.to(device)

criterion = nn.MarginRankingLoss(margin=1.0)
optimizer = optim.Adam(autoencoder.parameters(), lr=1e-4)
reg_lambda = 200
batch_num = 60

for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    train_loader.reset()
    for b in range(batch_num):
        batch, is_end = train_loader.generate_triple_batch(32)
        queries, docs1, docs2 = build_emb_input(embeddings, batch, QMAX_LEN, DMAX_LEN)
        
        optimizer.zero_grad()
        
        q_out = autoencoder(reshape2_4d(queries).to(device))
        d1_out = autoencoder(reshape2_4d(docs1).to(device))
        d2_out = autoencoder(reshape2_4d(docs2).to(device))
        reg_term = (
            torch.cat((q_out, d1_out, d2_out), dim=1)
            .sum(dim=1, keepdim=True)
            .to(device)
        )
        
        x1 = (q_out * d1_out).sum(dim=1, keepdim=True).to(device)
        x2 = (q_out * d2_out).sum(dim=1, keepdim=True).to(device)
        target = torch.ones(q_out.shape[0]).to(device)
        
        l1 = reg_lambda * reg_term
        loss = criterion(x1, x2, target) + l1
        print("L1: {}, loss: {}".format(l1.mean().item(), loss.mean().item()))
        loss.mean().backward()
        optimizer.step()
print('Finished Training')

Device to use: cuda:0
L1: 285589056.0, loss: 287600768.0
L1: 306297504.0, loss: 308180384.0
L1: 302670656.0, loss: 304625664.0
L1: 351272512.0, loss: 353987584.0
L1: 323707904.0, loss: 324397984.0
L1: 304095520.0, loss: 306171616.0
L1: 291535648.0, loss: 294242272.0
L1: 299635200.0, loss: 300983680.0
L1: 326978016.0, loss: 328638624.0
L1: 308871488.0, loss: 311057024.0
L1: 314361408.0, loss: 317998688.0
L1: 298604224.0, loss: 300580800.0
L1: 279983552.0, loss: 281040896.0
L1: 282396416.0, loss: 284478464.0
L1: 291824896.0, loss: 294367360.0
L1: 283633472.0, loss: 285278496.0
L1: 286950272.0, loss: 288598144.0
L1: 284055104.0, loss: 285492864.0
L1: 299348032.0, loss: 302558432.0
L1: 284441888.0, loss: 286830848.0
L1: 308829824.0, loss: 310629472.0
L1: 282067456.0, loss: 283229728.0
L1: 314054880.0, loss: 315758944.0
L1: 351926528.0, loss: 354868288.0
L1: 348356160.0, loss: 351802688.0
L1: 321465280.0, loss: 323791808.0
L1: 344139712.0, loss: 347106816.0
L1: 302761760.0, loss: 304583776.

L1: 261271968.0, loss: 263048512.0
L1: 244963808.0, loss: 246443936.0
L1: 256297920.0, loss: 258457344.0
L1: 250512960.0, loss: 252495744.0
L1: 227538976.0, loss: 229604000.0
L1: 243649152.0, loss: 246078080.0
L1: 219938048.0, loss: 221153232.0
L1: 234834080.0, loss: 236031552.0
L1: 231178976.0, loss: 232418624.0
L1: 271145792.0, loss: 273128512.0
L1: 251626320.0, loss: 252083536.0
L1: 236103488.0, loss: 237251840.0
L1: 226966512.0, loss: 228679808.0
L1: 230875232.0, loss: 231646416.0
L1: 252143520.0, loss: 253063584.0
L1: 238452400.0, loss: 240032208.0
L1: 241214976.0, loss: 243426400.0
L1: 229748864.0, loss: 230851104.0
L1: 217828320.0, loss: 218491008.0
L1: 220860480.0, loss: 222302720.0
L1: 223465392.0, loss: 224925344.0
L1: 220159872.0, loss: 221169568.0
L1: 223185504.0, loss: 224256864.0
L1: 219969072.0, loss: 220908864.0
L1: 229770512.0, loss: 231543200.0
L1: 220986592.0, loss: 222520704.0
L1: 239270592.0, loss: 240431200.0
L1: 220378944.0, loss: 221105472.0
L1: 241453056.0, los

L1: 184181520.0, loss: 184975696.0
L1: 185464880.0, loss: 186524848.0
L1: 188220608.0, loss: 188883904.0
L1: 201071488.0, loss: 202721088.0
L1: 194813568.0, loss: 195738064.0
L1: 204198464.0, loss: 205168544.0
L1: 192127424.0, loss: 193085808.0
L1: 202829952.0, loss: 204243456.0
L1: 199685728.0, loss: 200869408.0
L1: 182004256.0, loss: 183515200.0
L1: 193962320.0, loss: 195439056.0
L1: 174728064.0, loss: 175528032.0
L1: 187462416.0, loss: 188120368.0
L1: 183947712.0, loss: 184649632.0
L1: 216292512.0, loss: 217549632.0
L1: 200489408.0, loss: 200775936.0
L1: 190625344.0, loss: 191351008.0
L1: 182384192.0, loss: 183473696.0
L1: 185217168.0, loss: 185724784.0
L1: 202180192.0, loss: 202707200.0
L1: 189306592.0, loss: 190354784.0
L1: 191162048.0, loss: 192627760.0
L1: 183213152.0, loss: 183830240.0
L1: 172990528.0, loss: 173464064.0
L1: 176590208.0, loss: 177462288.0
L1: 178946752.0, loss: 179945952.0
L1: 176242704.0, loss: 176978816.0
L1: 176933840.0, loss: 177599168.0
L1: 175186432.0, los

L1: 149971936.0, loss: 150547168.0
L1: 157678384.0, loss: 158653648.0
L1: 155886688.0, loss: 156886496.0
L1: 156486336.0, loss: 157387392.0
L1: 150256672.0, loss: 150786320.0
L1: 149725968.0, loss: 150496064.0
L1: 152231168.0, loss: 152668096.0
L1: 162882400.0, loss: 164083104.0
L1: 158267328.0, loss: 158915776.0
L1: 165839696.0, loss: 166558608.0
L1: 156195616.0, loss: 156795712.0
L1: 164417920.0, loss: 165294944.0
L1: 162571872.0, loss: 163347136.0
L1: 148602544.0, loss: 149543472.0
L1: 158672896.0, loss: 159656928.0
L1: 143687104.0, loss: 144255264.0
L1: 152876080.0, loss: 153288512.0
L1: 150960704.0, loss: 151467808.0
L1: 178139312.0, loss: 179114176.0
L1: 163452672.0, loss: 163663744.0
L1: 155664544.0, loss: 156161792.0
L1: 150365440.0, loss: 151070336.0
L1: 150821392.0, loss: 151181168.0
L1: 165296624.0, loss: 165677520.0
L1: 153920352.0, loss: 154603264.0
L1: 155424416.0, loss: 156366928.0
L1: 149381664.0, loss: 149862976.0
L1: 142590592.0, loss: 142957600.0
L1: 144735104.0, los

L1: 137199840.0, loss: 138006608.0
L1: 141839616.0, loss: 142532880.0
L1: 136755136.0, loss: 137407744.0
L1: 130899504.0, loss: 131396448.0
L1: 127284656.0, loss: 127804176.0
L1: 125148208.0, loss: 125515768.0
L1: 130870984.0, loss: 131513448.0
L1: 129466768.0, loss: 130167432.0
L1: 130186176.0, loss: 130799600.0
L1: 125437648.0, loss: 125819744.0
L1: 124028368.0, loss: 124529232.0
L1: 127220752.0, loss: 127537552.0
L1: 134647488.0, loss: 135447168.0
L1: 130563120.0, loss: 130995984.0
L1: 135180464.0, loss: 135643760.0
L1: 129131120.0, loss: 129563280.0
L1: 137626528.0, loss: 138275616.0
L1: 136412256.0, loss: 136966016.0
L1: 123246248.0, loss: 123910832.0
L1: 131462816.0, loss: 132160688.0
L1: 119089992.0, loss: 119481512.0
L1: 126605272.0, loss: 126909712.0
L1: 125693328.0, loss: 126089376.0
L1: 146465296.0, loss: 147126368.0
L1: 137620960.0, loss: 137765888.0
L1: 130934072.0, loss: 131217936.0
L1: 125598568.0, loss: 126087712.0
L1: 125881648.0, loss: 126159464.0
L1: 138009440.0, los

L1: 105828072.0, loss: 106197232.0
L1: 98180480.0, loss: 98388496.0
L1: 104266128.0, loss: 104477312.0
L1: 107296816.0, loss: 107500624.0
L1: 111614640.0, loss: 111864024.0
L1: 115910224.0, loss: 116450832.0
L1: 118913208.0, loss: 119425048.0
L1: 115140192.0, loss: 115648400.0
L1: 109876768.0, loss: 110249624.0
L1: 107717560.0, loss: 108127152.0
L1: 106437000.0, loss: 106718056.0
L1: 110702240.0, loss: 111198128.0
L1: 108842496.0, loss: 109341984.0
L1: 109609600.0, loss: 110086360.0
L1: 105818360.0, loss: 106096848.0
L1: 106141320.0, loss: 106557016.0
L1: 106516416.0, loss: 106762024.0
L1: 114088832.0, loss: 114699064.0
L1: 110023840.0, loss: 110340248.0
L1: 114353944.0, loss: 114688936.0
L1: 108343960.0, loss: 108661360.0
L1: 115452144.0, loss: 115951032.0
L1: 114903344.0, loss: 115289112.0
L1: 104878104.0, loss: 105357736.0
L1: 112623184.0, loss: 113138496.0
L1: 101385200.0, loss: 101687776.0
L1: 107451464.0, loss: 107663712.0
L1: 106741792.0, loss: 107019872.0
L1: 123992200.0, loss:

L1: 84519056.0, loss: 84687840.0
L1: 89365152.0, loss: 89525984.0
L1: 90918992.0, loss: 91071856.0
L1: 95027208.0, loss: 95220976.0
L1: 98502608.0, loss: 98907136.0
L1: 101857528.0, loss: 102260080.0
L1: 97357952.0, loss: 97693152.0
L1: 94483856.0, loss: 94747160.0
L1: 92173664.0, loss: 92486320.0
L1: 90775160.0, loss: 91004200.0
L1: 94001920.0, loss: 94335552.0
L1: 92920832.0, loss: 93295504.0
L1: 94000000.0, loss: 94339600.0
L1: 89869008.0, loss: 90085848.0
L1: 89271296.0, loss: 89544304.0
L1: 91467840.0, loss: 91639776.0
L1: 97371072.0, loss: 97775808.0
L1: 94112816.0, loss: 94330752.0
L1: 95883632.0, loss: 96136592.0
L1: 93275384.0, loss: 93508792.0
L1: 98575120.0, loss: 98911432.0
L1: 99404688.0, loss: 99667584.0
L1: 90241104.0, loss: 90608704.0
L1: 96743184.0, loss: 97097184.0
L1: 86776656.0, loss: 86985568.0
L1: 91748512.0, loss: 91904464.0
L1: 91334744.0, loss: 91525792.0
L1: 107009568.0, loss: 107394584.0
L1: 100225840.0, loss: 100301696.0
L1: 96760208.0, loss: 96937872.0
L1: 

L1: 79202384.0, loss: 79391696.0
L1: 81296800.0, loss: 81553048.0
L1: 80817080.0, loss: 81092776.0
L1: 81055824.0, loss: 81289536.0
L1: 78698960.0, loss: 78867584.0
L1: 77562784.0, loss: 77780800.0
L1: 79596256.0, loss: 79742352.0
L1: 83531032.0, loss: 83841408.0
L1: 80060008.0, loss: 80243984.0
L1: 82635200.0, loss: 82826864.0
L1: 80585168.0, loss: 80756816.0
L1: 85594856.0, loss: 85848560.0
L1: 85930008.0, loss: 86152976.0
L1: 78177600.0, loss: 78451816.0
L1: 83554224.0, loss: 83823344.0
L1: 75597520.0, loss: 75767632.0
L1: 80220736.0, loss: 80338272.0
L1: 78677584.0, loss: 78815240.0
L1: 92858224.0, loss: 93175568.0
L1: 86754272.0, loss: 86812272.0
L1: 83859648.0, loss: 83984392.0
L1: 79920736.0, loss: 80120016.0
L1: 79139688.0, loss: 79259560.0
L1: 87834472.0, loss: 87962712.0
L1: 80931208.0, loss: 81168928.0
L1: 80736640.0, loss: 81046528.0
L1: 78507328.0, loss: 78656752.0
L1: 75021120.0, loss: 75140384.0
L1: 76659512.0, loss: 76828872.0
L1: 77151088.0, loss: 77377264.0
L1: 770449

In [87]:
for epoch in range(4*epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    train_loader.reset()
    for b in range(batch_num):
        batch, is_end = train_loader.generate_triple_batch(32)
        queries, docs1, docs2 = build_emb_input(embeddings, batch, QMAX_LEN, DMAX_LEN)
        
        optimizer.zero_grad()
        
        q_out = autoencoder(reshape2_4d(queries).to(device))
        d1_out = autoencoder(reshape2_4d(docs1).to(device))
        d2_out = autoencoder(reshape2_4d(docs2).to(device))
        reg_term = (
            torch.cat((q_out, d1_out, d2_out), dim=1)
            .sum(dim=1, keepdim=True)
            .to(device)
        )
        
        x1 = (q_out * d1_out).sum(dim=1, keepdim=True).to(device)
        x2 = (q_out * d2_out).sum(dim=1, keepdim=True).to(device)
        target = torch.ones(q_out.shape[0]).to(device)
        
        l1 = reg_lambda * reg_term
        loss = criterion(x1, x2, target) + l1
        print("L1: {}, loss: {}".format(l1.mean().item(), loss.mean().item()))
        loss.mean().backward()
        optimizer.step()
print('Finished Training')

L1: 10240407.0, loss: 10244572.0
L1: 10957500.0, loss: 10961192.0
L1: 10821942.0, loss: 10825017.0
L1: 12552675.0, loss: 12559575.0
L1: 11513247.0, loss: 11514667.0
L1: 10893610.0, loss: 10898064.0
L1: 10346368.0, loss: 10352054.0
L1: 10582967.0, loss: 10585632.0
L1: 11554101.0, loss: 11556337.0
L1: 11103068.0, loss: 11106860.0
L1: 11197121.0, loss: 11203859.0
L1: 10705030.0, loss: 10708498.0
L1: 10096574.0, loss: 10098955.0
L1: 10142702.0, loss: 10147459.0
L1: 10509820.0, loss: 10514973.0
L1: 10098859.0, loss: 10102074.0
L1: 10329764.0, loss: 10333140.0
L1: 10206072.0, loss: 10208708.0
L1: 10715460.0, loss: 10721772.0
L1: 10317528.0, loss: 10322253.0
L1: 11024684.0, loss: 11027540.0
L1: 10167382.0, loss: 10169681.0
L1: 11604349.0, loss: 11607841.0
L1: 12847628.0, loss: 12853408.0
L1: 12660940.0, loss: 12667212.0
L1: 11785755.0, loss: 11790706.0
L1: 12689264.0, loss: 12695122.0
L1: 11124474.0, loss: 11128404.0
L1: 10977455.0, loss: 10980560.0
L1: 10861585.0, loss: 10863876.0
L1: 105947

L1: 8615749.0, loss: 8617845.0
L1: 8846102.0, loss: 8848348.0
L1: 8745050.0, loss: 8746764.0
L1: 9174854.0, loss: 9179142.0
L1: 8847799.0, loss: 8851153.0
L1: 9425309.0, loss: 9427163.0
L1: 8691854.0, loss: 8693434.0
L1: 10015224.0, loss: 10017634.0
L1: 11084542.0, loss: 11088509.0
L1: 10901150.0, loss: 10905609.0
L1: 10113822.0, loss: 10117204.0
L1: 10949660.0, loss: 10953624.0
L1: 9557292.0, loss: 9559978.0
L1: 9408749.0, loss: 9410777.0
L1: 9329682.0, loss: 9331012.0
L1: 9094429.0, loss: 9095568.0
L1: 8384421.0, loss: 8386893.5
L1: 8947150.0, loss: 8949754.0
L1: 8495579.0, loss: 8497210.0
L1: 8697626.0, loss: 8699454.0
L1: 8675183.0, loss: 8678516.0
L1: 7998293.5, loss: 8000155.0
L1: 8504090.0, loss: 8505312.0
L1: 8799710.0, loss: 8800532.0
L1: 9337403.0, loss: 9339349.0
L1: 9875910.0, loss: 9882262.0
L1: 10100270.0, loss: 10103172.0
L1: 9757206.0, loss: 9761110.0
L1: 8994922.0, loss: 8997796.0
L1: 8663338.0, loss: 8666306.0
L1: 8675171.0, loss: 8677259.0
L1: 9461194.0, loss: 946544

L1: 8341569.5, loss: 8345917.5
L1: 8517708.0, loss: 8519485.0
L1: 8227462.0, loss: 8229984.5
L1: 7558003.5, loss: 7559869.5
L1: 7267237.5, loss: 7269012.5
L1: 7294266.0, loss: 7295541.0
L1: 8001302.0, loss: 8004091.0
L1: 7927908.5, loss: 7930880.5
L1: 8092981.0, loss: 8096171.0
L1: 7693637.0, loss: 7694835.0
L1: 7652215.0, loss: 7654398.0
L1: 7827999.5, loss: 7829142.5
L1: 8343360.0, loss: 8347240.5
L1: 7971196.5, loss: 7972777.0
L1: 8547478.0, loss: 8549470.0
L1: 7932575.0, loss: 7934712.0
L1: 8333184.5, loss: 8335844.5
L1: 8098611.0, loss: 8101427.0
L1: 7360824.0, loss: 7363765.0
L1: 7791892.0, loss: 7795253.0
L1: 7085827.0, loss: 7087445.0
L1: 7584818.5, loss: 7586452.5
L1: 7494311.5, loss: 7495473.0
L1: 8679023.0, loss: 8681473.0
L1: 7925803.0, loss: 7926288.5
L1: 7470296.0, loss: 7472391.0
L1: 7057221.0, loss: 7059543.0
L1: 7282507.0, loss: 7283412.0
L1: 7931828.5, loss: 7932614.5
L1: 7694328.0, loss: 7695592.0
L1: 7772941.5, loss: 7775557.0
L1: 7402797.0, loss: 7404221.0
L1: 6961

L1: 6152180.0, loss: 6153620.0
L1: 5794918.0, loss: 5796382.0
L1: 5995543.5, loss: 5996073.5
L1: 6526928.0, loss: 6527389.0
L1: 6348723.0, loss: 6349466.5
L1: 6415199.5, loss: 6416820.0
L1: 6104053.5, loss: 6104938.5
L1: 5734989.0, loss: 5735513.0
L1: 5722014.0, loss: 5723041.0
L1: 5994326.0, loss: 5995765.0
L1: 5674965.5, loss: 5675644.0
L1: 5876109.0, loss: 5876878.0
L1: 5809832.5, loss: 5810436.0
L1: 6069032.5, loss: 6070603.0
L1: 5879313.5, loss: 5880655.5
L1: 6226512.0, loss: 6227108.0
L1: 5739634.0, loss: 5740241.0
L1: 6756865.0, loss: 6757767.0
L1: 7462517.0, loss: 7464015.0
L1: 7289190.0, loss: 7291030.0
L1: 6731941.0, loss: 6733233.0
L1: 7373676.0, loss: 7375173.5
L1: 6378757.5, loss: 6379870.0
L1: 6242535.0, loss: 6243233.0
L1: 6219060.5, loss: 6219461.0
L1: 6053418.5, loss: 6053858.5
L1: 5526890.5, loss: 5527876.0
L1: 5923388.0, loss: 5924343.0
L1: 5593017.0, loss: 5593600.0
L1: 5761412.0, loss: 5762015.0
L1: 5720273.0, loss: 5721476.0
L1: 5268098.0, loss: 5268823.0
L1: 5602

L1: 4448806.0, loss: 4449421.0
L1: 4778879.0, loss: 4779466.5
L1: 4500518.0, loss: 4500872.5
L1: 4639755.5, loss: 4640109.0
L1: 4603909.5, loss: 4604622.0
L1: 4240961.5, loss: 4241405.5
L1: 4505449.0, loss: 4505749.0
L1: 4682151.0, loss: 4682253.0
L1: 4966647.0, loss: 4967102.0
L1: 5298994.0, loss: 5300580.0
L1: 5387633.0, loss: 5388184.0
L1: 5201680.0, loss: 5202488.5
L1: 4759837.5, loss: 4760500.5
L1: 4573809.0, loss: 4574292.5
L1: 4609539.5, loss: 4609949.5
L1: 5103027.0, loss: 5103906.0
L1: 5046146.0, loss: 5047121.0
L1: 5192557.0, loss: 5193831.0
L1: 4938757.0, loss: 4939106.5
L1: 4849122.0, loss: 4849913.0
L1: 4976549.0, loss: 4976936.5
L1: 5313622.0, loss: 5315020.0
L1: 5014373.0, loss: 5014854.0
L1: 5439934.0, loss: 5440590.5
L1: 5031388.5, loss: 5032200.5
L1: 5275132.0, loss: 5276084.0
L1: 5122220.0, loss: 5123241.0
L1: 4664007.0, loss: 4664998.0
L1: 4923248.0, loss: 4924475.0
L1: 4481195.5, loss: 4481738.5
L1: 4789916.0, loss: 4790517.5
L1: 4729803.0, loss: 4730181.0
L1: 5478

...

L1: 32687.515625, loss: 32688.234375
L1: 32541.146484375, loss: 32541.65625
L1: 30519.59375, loss: 30520.423828125
L1: 30637.61328125, loss: 30638.44140625
L1: 31691.27734375, loss: 31691.951171875
L1: 28435.728515625, loss: 28436.369140625
L1: 33069.9921875, loss: 33070.828125
L1: 36120.98828125, loss: 36121.86328125
L1: 36073.1015625, loss: 36073.8515625
L1: 28800.05078125, loss: 28800.861328125
L1: 27861.390625, loss: 27862.07421875
L1: 30934.81640625, loss: 30935.77734375
L1: 31039.42578125, loss: 31040.140625
L1: 29270.11328125, loss: 29270.869140625
L1: 28039.55078125, loss: 28040.36328125
L1: 30780.126953125, loss: 30780.962890625
L1: 27469.6171875, loss: 27470.3671875
L1: 25565.703125, loss: 25566.52734375
L1: 29226.16015625, loss: 29226.91015625
L1: 31174.3828125, loss: 31175.140625
L1: 32420.69921875, loss: 32421.263671875
L1: 34691.01953125, loss: 34691.7421875
L1: 38356.3046875, loss: 38357.08984375
L1: 29961.69921875, loss: 29962.5859375
L1: 33533.19921875, loss: 33533.835

L1: 27309.49609375, loss: 27310.20703125
L1: 28997.9921875, loss: 28998.830078125
L1: 30189.126953125, loss: 30189.65234375
L1: 35365.78515625, loss: 35366.36328125
L1: 33066.34375, loss: 33067.140625
L1: 26268.578125, loss: 26269.33203125
L1: 31694.052734375, loss: 31694.701171875
L1: 26166.013671875, loss: 26166.83984375
L1: 24423.921875, loss: 24424.640625
L1: 28023.232421875, loss: 28023.779296875
L1: 27620.16796875, loss: 27621.044921875
L1: 29474.9921875, loss: 29475.5546875
L1: 26928.3515625, loss: 26929.029296875
L1: 31584.6015625, loss: 31585.140625
L1: 27234.69140625, loss: 27235.65234375
L1: 28001.9609375, loss: 28002.62109375
L1: 24704.61328125, loss: 24705.37890625
L1: 26413.21875, loss: 26413.90234375
L1: 30481.263671875, loss: 30482.033203125
L1: 28172.28515625, loss: 28173.1328125
L1: 24249.75, loss: 24250.46875
L1: 22886.794921875, loss: 22887.779296875
L1: 23224.560546875, loss: 23225.240234375
L1: 29120.865234375, loss: 29121.6328125
L1: 22982.115234375, loss: 22982.

L1: 25068.0390625, loss: 25068.720703125
L1: 22194.10546875, loss: 22194.927734375
L1: 20408.763671875, loss: 20409.53515625
L1: 26491.24609375, loss: 26492.04296875
L1: 27286.052734375, loss: 27286.990234375
L1: 26140.74609375, loss: 26141.490234375
L1: 22758.814453125, loss: 22759.62890625
L1: 24394.38671875, loss: 24395.1328125
L1: 25464.78125, loss: 25465.58203125
L1: 24533.052734375, loss: 24533.806640625
L1: 24425.12890625, loss: 24425.7109375
L1: 22611.7265625, loss: 22612.5703125
L1: 22705.78515625, loss: 22706.6640625
L1: 23558.1015625, loss: 23558.841796875
L1: 21000.02734375, loss: 21000.806640625
L1: 24769.142578125, loss: 24769.955078125
L1: 27118.0703125, loss: 27118.8671875
L1: 27139.779296875, loss: 27140.55859375
L1: 21190.236328125, loss: 21191.09765625
L1: 20470.578125, loss: 20471.33984375
L1: 22803.5, loss: 22804.455078125
L1: 22618.85546875, loss: 22619.640625
L1: 21987.771484375, loss: 21988.546875
L1: 20862.73828125, loss: 20863.623046875
L1: 22650.998046875, lo

L1: 19143.966796875, loss: 19144.759765625
L1: 20380.5, loss: 20381.265625
L1: 20959.53515625, loss: 20960.22265625
L1: 22883.640625, loss: 22884.41015625
L1: 25718.48046875, loss: 25719.2421875
L1: 19444.890625, loss: 19445.79296875
L1: 22671.13671875, loss: 22671.86328125
L1: 20528.9921875, loss: 20529.77734375
L1: 21164.11328125, loss: 21165.0234375
L1: 22302.623046875, loss: 22303.240234375
L1: 26619.75, loss: 26620.365234375
L1: 24929.234375, loss: 24930.03515625
L1: 19567.01171875, loss: 19567.830078125
L1: 23367.056640625, loss: 23367.8125
L1: 19533.0859375, loss: 19533.943359375
L1: 18195.69140625, loss: 18196.484375
L1: 20912.8828125, loss: 20913.50390625
L1: 20816.3359375, loss: 20817.27734375
L1: 22009.42578125, loss: 22010.083984375
L1: 20127.646484375, loss: 20128.416015625
L1: 24022.8671875, loss: 24023.4921875
L1: 20209.92578125, loss: 20210.85546875
L1: 21107.537109375, loss: 21108.271484375
L1: 18435.78515625, loss: 18436.60546875
L1: 19884.974609375, loss: 19885.74023

L1: 20211.638671875, loss: 20212.470703125
L1: 18913.76171875, loss: 18914.705078125
L1: 15715.0048828125, loss: 15715.8291015625
L1: 14770.267578125, loss: 14771.236328125
L1: 14963.6708984375, loss: 14964.4580078125
L1: 18963.943359375, loss: 18964.771484375
L1: 14787.6533203125, loss: 14788.5458984375
L1: 17284.486328125, loss: 17285.205078125
L1: 18939.796875, loss: 18940.58984375
L1: 16607.4765625, loss: 16608.3125
L1: 15018.74609375, loss: 15019.537109375
L1: 20158.73046875, loss: 20159.53125
L1: 20510.431640625, loss: 20511.357421875
L1: 19908.734375, loss: 19909.50390625
L1: 17186.99609375, loss: 17187.849609375
L1: 18220.263671875, loss: 18221.072265625
L1: 19161.98828125, loss: 19162.8203125
L1: 18462.611328125, loss: 18463.396484375
L1: 18477.126953125, loss: 18477.794921875
L1: 16869.21484375, loss: 16870.07421875
L1: 16990.42578125, loss: 16991.296875
L1: 17628.861328125, loss: 17629.666015625
L1: 15627.5498046875, loss: 15628.408203125
L1: 18668.80859375, loss: 18669.5976

In [88]:
def evaluate_repr(embeddings, autoencoder, batch, input_type):
    max_len = QMAX_LEN if input_type == "queries" else DMAX_LEN
    emb = embeddings.matrix2(batch, max_len=max_len)

    autoencoder.eval()
    d_out = autoencoder(reshape2_4d(emb).to(device))

    return d_out[:, :, 0].detach()


In [89]:
def zeros(x):
    return len([i for i, e in enumerate(x) if e == 0])

def nans(x):
    return len([i for i, e in enumerate(x) if np.isnan(e)])

def get_zeros(queries, docs):
    q_out = evaluate_repr(embeddings, autoencoder, queries, "queries").detach().cpu()
    d_out = evaluate_repr(embeddings, autoencoder, docs, "docs").detach().cpu()
    for i in range(queries.shape[0]):
        print("Iteration #"+str(i)+ ":")
        print("Zeros in query: {}, len = {}".format(zeros(q_out[i]), len(queries[i].split(" "))))
        print("Zeros in doc: {}".format(zeros(d_out[i])))
        
    #for i in range(queries.shape[0]):
    #    print("Iteration #"+str(i)+ ":")
    #    print("Nans in query: ", nans(q_out[i]))
    #    print("Nans in doc: ", nans(d_out[i]))


In [90]:
valid_loader.reset()
_, docs_batch, is_end = valid_loader.generate_docs(20)
_, queries_batch, is_end = valid_loader.generate_queries(20)
get_zeros(queries_batch, docs_batch)

Iteration #0:
Zeros in query: 9995, len = 10
Zeros in doc: 9577
Iteration #1:
Zeros in query: 9998, len = 8
Zeros in doc: 9648
Iteration #2:
Zeros in query: 9976, len = 2
Zeros in doc: 9842
Iteration #3:
Zeros in query: 9981, len = 8
Zeros in doc: 9802
Iteration #4:
Zeros in query: 9961, len = 5
Zeros in doc: 9252
Iteration #5:
Zeros in query: 9930, len = 6
Zeros in doc: 9770
Iteration #6:
Zeros in query: 9964, len = 3
Zeros in doc: 9901
Iteration #7:
Zeros in query: 9962, len = 5
Zeros in doc: 9727
Iteration #8:
Zeros in query: 9975, len = 8
Zeros in doc: 9789
Iteration #9:
Zeros in query: 10000, len = 7
Zeros in doc: 9644
Iteration #10:
Zeros in query: 9957, len = 4
Zeros in doc: 9708
Iteration #11:
Zeros in query: 10000, len = 16
Zeros in doc: 9751
Iteration #12:
Zeros in query: 9975, len = 6
Zeros in doc: 9677
Iteration #13:
Zeros in query: 9959, len = 4
Zeros in doc: 9775
Iteration #14:
Zeros in query: 9985, len = 9
Zeros in doc: 9742
Iteration #15:
Zeros in query: 9949, len = 5
Z

In [91]:
torch.save(autoencoder.state_dict(), "separate_model.pth")

In [94]:
from utils.evaluation_helpers import *

eval_loader = utils.msmarco.data_loader.DataLoader(
    test_queries, test_qrels, docs_dict,
)

In [103]:
import json
import numpy as np
from datetime import datetime


def estimate_sparsity(repres):
    zero = 0
    for i in range(len(repres)):
        if repres[i] == 0.0:
            zero += 1
    return zero

class InvertedIndex:
    def __init__(self, out_file):
        self.index = dict()
        self.out_file = out_file

    def construct(self, doc_ids, doc_repres):
        counter = 0
        repres_len = doc_repres.shape[1]
        for i in range(doc_repres.shape[0]):
            if counter < 3:
                print("Document zero elements: ", estimate_sparsity(doc_repres[i]), len(doc_repres[i]), flush=True)    
            counter += 1
            for j in range(repres_len):
                if doc_repres[i][j] > 0.0:
                    if j not in self.index:
                        self.index[j] = []
                    self.index[j].append([str(doc_ids[i]), doc_repres[i][j].item()])

    def get_index(self):
        return self.index

    def flush(self):
        json_file = json.dumps(self.index)
        f = open(self.out_file, "w")
        f.write(json_file)
        f.close()

    def read_index(self):
        with open(self.out_file, "r") as f:
            self.index = json.load(f)
        self.index = {k: v for k, v in self.index.items()}
        return self.index


"""
    Building an inverted index:
    0 - [(doc_id, weight), ...]
    1 - [(doc_id, weight), ...]
    2
    .
    .
    .
    300

    Model should already be trained at this step.
    We take each document and evaluate its representation by the trained model.
    Then according to the representation the inverted index is built.
"""


def build_inverted_index(batch_size, model, eval_loader, iidx_file, dump=False):
    print("Building inverted index started...", flush = True)
    start = datetime.now()
    inverted_index = InvertedIndex(iidx_file)
    is_end = False
    while not is_end:
        s = datetime.now()
        doc_ids, docs, is_end = eval_loader.generate_docs(batch_size)
        repr = evaluate_repr(embeddings, model, docs, input_type="docs")
        inverted_index.construct(doc_ids, repr)
        print("Inv.index for one batch for time: {}".format(datetime.now()-s), flush=True)
    if dump:
        inverted_index.flush()
    time = datetime.now() - start
    print("Inverted index is built! Time: ", time, flush = True)
    return inverted_index


def load_inverted_index(filename):
    print("Loading existing inverted index from {}.".format(filename))
    index = InvertedIndex(filename)
    index.read_index()
    return index


In [ ]:
index = build_inverted_index(32,
            autoencoder,
            valid_loader,
            "inv_ind.txt",
            dump=True,
        )

Building inverted index started...
Document zero elements:  9875 10000
Document zero elements:  9670 10000
Document zero elements:  9701 10000
Inv.index for one batch for time: 0:00:15.209001
Document zero elements:  9498 10000
Document zero elements:  9801 10000
Document zero elements:  9844 10000
Inv.index for one batch for time: 0:00:15.293310
Document zero elements:  9399 10000
Document zero elements:  9559 10000
Document zero elements:  9456 10000
Inv.index for one batch for time: 0:00:14.987838
Document zero elements:  9625 10000
Document zero elements:  9454 10000
Document zero elements:  9724 10000
Inv.index for one batch for time: 0:00:20.169740
Document zero elements:  9782 10000
Document zero elements:  9529 10000
Document zero elements:  9708 10000
Inv.index for one batch for time: 0:00:15.176344
Document zero elements:  9769 10000
Document zero elements:  9622 10000
Document zero elements:  9647 10000
Inv.index for one batch for time: 0:00:20.158219
Document zero elements:

In [ ]:
import json
from .helpers import dump
from datetime import datetime

class RetrievalScore:
    def __init__(self):
        self.retrieval_score = None
        self.is_evaluated = False

    def estimate_sparsity(self, repres):
        zero = 0
        for i in range(len(repres)):
            if repres[i] == 0.0:
                zero += 1
        return zero

    """
        This function returns a dictionary, where each query has a correspondent
        relative documents with the corresponding score.
    """

    def evaluate(self, eval_loader, index, model, batch_size):
        start = datetime.now()
        if self.is_evaluated:
            return self.retrieval_score
        is_end = False
        self.retrieval_score = dict()
        counter = 0
        while not is_end:
            queries_id, queries, is_end = eval_loader.generate_queries(batch_size)
            qreprs = model.evaluate_repr(queries, input_type="queries")
            for qrepr, q in zip(qreprs, queries_id):
                if counter < 3:
                    print("Zero elements: ", self.estimate_sparsity(qrepr), len(qrepr), flush=True)
                counter += 1
                self.retrieval_score[str(q)] = self.__retrieval_score_for_query(
                    qrepr, index
                )  # returns dict({doc_id:val})
        time = datetime.now() - start
        print("Retrieval score is built for ", time, flush = True)
        self.is_evaluated = True
        return self.retrieval_score

    """
        This function estimates retrieval score for each query
        over all possible documents from the inverted index.

        Returns a dictionaty of shape:
        {
            'doc1_id': val,
            'doc2_id': val
        }
    """

    def __retrieval_score_for_query(self, query_repr, index):
        relevant_docs = dict()
        for i in range(len(query_repr)):
            if query_repr[i] != 0.0:
                try:
                    docs = index.get_index()[i]
                except KeyError:
                    continue
                docs = index.get_index()[i]
                for j in range(len(docs)):
                    doc_id = str(docs[j][0])
                    if doc_id not in relevant_docs:
                        relevant_docs[doc_id] = query_repr[i].item() * docs[j][1]
                    else:
                        relevant_docs[doc_id] += query_repr[i].item() * docs[j][1]
        return relevant_docs

    """
        Dump retrieval score.
    """

    def dump(self, filename):
        if not self.is_evaluated:
            print("Retrieval score is not yet evaluated")
        else:
            dump(self.retrieval_score, filename)

    """
        Read retrieval score to dict.
    """

    def read(self, filename):
        if self.is_evaluated:
            print("One retrieval score is already loaded")
        else:
            with open(filename, "r") as f:
                self.retrieval_score = json.load(f)
        return self.retrieval_score

In [ ]:
def retr_score(model, batch_size, eval_loader, index)
    retrieval_score = RetrievalScore()
    predicted_qrels = retrieval_score.evaluate(
        eval_loader, index, model, batch_size
    )
    if dump:
        print("Dumping retrieval_score to ", model_params["retrieval_score"])
        retrieval_score.dump(model_params["retrieval_score"])
    return predicted_qrels

In [ ]:
pred_qrels = retr_score(autoencoder, 32, eval_loader, index)

In [ ]:
from utils.evaluation_helpers import _evaluate_metrics
metrics = _evaluate_metrics(predicted_qrels, eval_loader.generate_qrels(),  [
        "P_5",
        "P_10",
        "P_20",
        "ndcg_cut_5",
        "ndcg_cut_10",
        "ndcg_cut_20",
        "ndcg_cut_1000",
        "map",
        "recall_1000"
    ], "final_metrics.txt", True)